In [0]:
df = spark.createDataFrame(
    [([["a","b","c"], ["d","e","f"], ["g","h","i", "j"]],)],
    ["data"])
df.show(20,truncate=False)

+------------------------------------+
|data                                |
+------------------------------------+
|[[a, b, c], [d, e, f], [g, h, i, j]]|
+------------------------------------+



In [0]:
from pyspark.sql.functions import *
df = df.withColumn("data1",explode("data"))
df.select('data1').show()

+------------+
|       data1|
+------------+
|   [a, b, c]|
|   [d, e, f]|
|[g, h, i, j]|
+------------+



In [0]:
max_size = df.select(max(size('data1'))).collect()[0][0]

df.select(
    *[col("data1")[i].alias(f"col_{i}") for i in range(max_size)]
).show()

+-----+-----+-----+-----+
|col_0|col_1|col_2|col_3|
+-----+-----+-----+-----+
|    a|    b|    c| null|
|    d|    e|    f| null|
|    g|    h|    i|    j|
+-----+-----+-----+-----+



In [0]:
walmart=spark.read.csv("/FileStore/tables/Walmart.csv",header=True,inferSchema=True)
walmart.show()

+-----+----------+------------+------------+-----------+----------+-----------+------------+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|
+-----+----------+------------+------------+-----------+----------+-----------+------------+
|    1|05-02-2010|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|
|    1|12-02-2010|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|
|    1|19-02-2010|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|
|    1|26-02-2010|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|
|    1|05-03-2010|  1554806.68|           0|       46.5|     2.625|211.3501429|       8.106|
|    1|12-03-2010|  1439541.59|           0|      57.79|     2.667|211.3806429|       8.106|
|    1|19-03-2010|  1472515.79|           0|      54.58|      2.72| 211.215635|       8.106|
|    1|26-03-2010|  1404429.92|           0|      51.45|     2.732|211

In [0]:
# Q) which store has the highest weekly sales on which date
from pyspark.sql.functions import *
walmart.select("Store","Date","Weekly_Sales").orderBy(desc("Weekly_Sales")).limit(1).show()

+-----+----------+------------+
|Store|      Date|Weekly_Sales|
+-----+----------+------------+
|   14|24-12-2010|  3818686.45|
+-----+----------+------------+



In [0]:
from pyspark.sql.functions import *
walmart = walmart.withColumn('year',year(to_timestamp('Date', 'dd-MM-yyyy')))
walmart.show()

+-----+----------+------------+------------+-----------+----------+-----------+------------+----+
|Store|      Date|Weekly_Sales|Holiday_Flag|Temperature|Fuel_Price|        CPI|Unemployment|year|
+-----+----------+------------+------------+-----------+----------+-----------+------------+----+
|    1|05-02-2010|   1643690.9|           0|      42.31|     2.572|211.0963582|       8.106|2010|
|    1|12-02-2010|  1641957.44|           1|      38.51|     2.548|211.2421698|       8.106|2010|
|    1|19-02-2010|  1611968.17|           0|      39.93|     2.514|211.2891429|       8.106|2010|
|    1|26-02-2010|  1409727.59|           0|      46.63|     2.561|211.3196429|       8.106|2010|
|    1|05-03-2010|  1554806.68|           0|       46.5|     2.625|211.3501429|       8.106|2010|
|    1|12-03-2010|  1439541.59|           0|      57.79|     2.667|211.3806429|       8.106|2010|
|    1|19-03-2010|  1472515.79|           0|      54.58|      2.72| 211.215635|       8.106|2010|
|    1|26-03-2010|  

In [0]:
#What is the max Weekly_Sales per year?

walmart.groupBy("year").max("Weekly_Sales").show()

+----+-----------------+
|year|max(Weekly_Sales)|
+----+-----------------+
|2012|       2565259.92|
|2010|       3818686.45|
|2011|       3676388.98|
+----+-----------------+

